In [1]:
###Functions that generate simulated data, and split the simulated data into train, cal and test.
import numpy as np
from sympy import symbols, Eq, solve


def find_beta(d,R_squared,error):
    """
    input:
    d: The dimension of X, e.g., d=1000
    R_squared: 
    error: homo or hetero
    
    output:
    beta: a scaler
    """
    assert error in ['homo','hetero'], 'error must be homo or hetero!'
    total_sum=0
    for j in range(2, d+1):
        term = 1 / (j**2)
        total_sum += term
    
    beta=symbols('beta')
    if error=='homo':
        equation=Eq(total_sum*beta**2-R_squared*(total_sum*beta**2+1),0)
        solutions=solve(equation,beta)
        beta_hat=[sol.evalf() for sol in solutions if sol > 0]
    else:
        equation=Eq(total_sum*beta**2-R_squared*(total_sum*beta**2+35),0)
        solutions=solve(equation,beta)
        beta_hat=[sol.evalf() for sol in solutions if sol > 0]
    
    return beta_hat

def generate_data(n,d,R_squared,error,mean):
    """
    n: sample size
    d: dimensionality of X, e.g, d=1000
    R_squared: 
    error: homo or hetero
    mean:
    
    """
    assert error in ['homo','hetero'], 'error must be homo or hetero!'
    #mean=np.full(d,mean)
    #cov=np.eye(d)
    #X=np.random.multivariate_normal(mean,cov,size=n)
    X=np.random.normal(mean,1,(n,d))
    X[:,0]=1

    beta_hat=find_beta(d,R_squared,error)
    coef=[1/(j+1) for j in range(d)]
    coef=np.array(coef)
    
    coef=coef*beta_hat
    Y_reg=X.dot(coef)
    
    if error=='homo':
        eps=np.random.normal(loc=0,scale=1,size=n)
    else:
        eps0=np.random.normal(loc=0,scale=1,size=n)
        X_sub=X[:,1:6]
        hetero_x_squared=np.sum(X_sub**2,axis=1)
        eps=hetero_x_squared*eps0
    
    Y=Y_reg+eps
    
    data={'X':X,'Y':Y}
    return data
        
def split_data(data,n0,n1,n,d_set='train'):
    """
    n0:train data
    n1: calibration data
    n:test data
    """
    assert d_set in ['train', 'cal','test'], 'd_set must be train, cal, or test.'
    
    if d_set=='train':
        X=data['X'][:n0]
        Y=data['Y'][:n0]
    elif d_set=='cal':
        X=data['X'][n0:(n0+n1)]
        Y=data['Y'][n0:(n0+n1)]
    else:
        X=data['X'][n0+n1:]
        Y=data['Y'][n0+n1:]
        
    data_split={'X':X,'Y':Y}
    return data_split
        

In [2]:
from sklearn.linear_model import LinearRegression
import copy
def fit_candidate_model(X_train,Y_train,M):
    """
    input: 
    X_train: n*d dimensiolal 
    Y_train: n dim
    M: the number of candidate models
    
    output:
    M candidate trained models
    """
    candidate_models=[]
    for i in range(M):
        X_train_candidate=copy.deepcopy(X_train[:,1:i+2])
        trained_model = LinearRegression(n_jobs=2).fit(X_train_candidate,Y_train)
        candidate_models.append(trained_model)
    
    return candidate_models

def jackknife_prediction(X_train,Y_train,M):
    """
    input: 
    X_train: n*d dimensiolal array
    Y_train: n dim array
    M: the number of candidate models
    
    output:
    prediction_jackknife: n_train*M dimensional array
    """
    n_train=len(Y_train)
    prediction_jackknife=np.zeros((M,n_train))
    
    for i in range(M):
        X_train_candidate=copy.deepcopy(X_train[:,1:i+2])
        for j in range(n_train):
            X_train_jackknife=np.delete(X_train_candidate,j,axis=0)
            Y_train_jackknife=np.delete(Y_train,j)
            trained_model = LinearRegression(n_jobs=2).fit(X_train_jackknife,Y_train_jackknife)
            
            X_test_jackknife=X_train_candidate[j,:].reshape(1, -1)
            prediction_jackknife[i,j]=trained_model.predict(X_test_jackknife)
    
    prediction_jackknife=prediction_jackknife.T
    return prediction_jackknife

            

In [3]:
###Function that find the optimal weights
from qpsolvers import solve_qp
import scipy.sparse as sp
def weights_criterion_jackknife(Y_train,prediction_jackknife,kernel_weights,candidate_interval_lengths,penalty_coef,criterion):
    """
    input:
    Y_train: n dimensional array
    prediction_jackknife: n*M dim
    kernel_weights: n dimensioal array
    candidate_interval_lengths: M dimensional array
    penalty_coef: i.e.,lambda, a scalar
    criterion: ordinary, weighted, or weighted_plus_penalty
    
    output:
    optimal_weights: M dimensional array
    
    """
    assert criterion in ['ordinary','weighted','weighted_plus_penalty'], 'criterion must be ordinary, weighted, or weighted_plus_penalty!'
    
    n_train=prediction_jackknife.shape[0]
    M=prediction_jackknife.shape[1]
    A=np.ones(M).reshape((M,))
    A=sp.csc_matrix(A)
    b = np.array([1], dtype=np.double)
    lb=np.zeros(M).reshape((M,))
    ub=np.ones(M).reshape((M,))
    
    if criterion=='ordinary':
        P_kernel=np.eye(n_train)
        penalty_coef=0
        L=np.zeros((M,M))
    elif criterion=='weighted':
        assert kernel_weights is not None, 'kernel_weights must be specified!'
        P_kernel=np.diag(kernel_weights)
        penalty_coef=0
        L=np.zeros((M,M))
    else:
        assert kernel_weights is not None, 'kernel_weights must be specified!'
        assert penalty_coef is not None, 'penalty_coef must be specified!'
        assert candidate_interval_lengths is not None, 'candidate_interval_lengths must be specified!'
        P_kernel=np.diag(kernel_weights)
        penalty_coef=penalty_coef
        L=np.diag(candidate_interval_lengths)
    
    P=2*(prediction_jackknife.T.dot(P_kernel).dot(prediction_jackknife)+penalty_coef*L.T.dot(L))
    P=sp.csc_matrix(P)
    q=-2*prediction_jackknife.T.dot(P_kernel).dot(Y_train)
    optimal_weights=solve_qp(P=P, q=q, A=A, b=b,lb=lb,ub=ub,solver="scs")
    
    return optimal_weights
    

def find_kernel_weights(X,x_datapoint,h):
    """
    input:
    X: n*d dimensional array
    x_datapoint: a single data point, d dimensional array
    h: bandwidth
    
    output:
    kernel_weights: n dimensional array
    """
    n_X=X.shape[0]
    d=X.shape[1]
    distances=np.zeros(n_X)



    """
    X=X[:,0:6]
    x_datapoint=x_datapoint[0:6]
    
    coef=[1/(j+1) for j in range(d)]
    coef=np.array(coef)
    X=X*coef
    x_datapoint=x_datapoint*coef  


    """

    
    for i in range(n_X):
        distances[i] = np.linalg.norm(x_datapoint - X[i,],ord=2)**2
    
    kernel=np.exp(-distances/(2*h**2))
    kernel_weights=kernel/np.sum(kernel)
    
    return kernel_weights

In [4]:
def quantile_corrected(x,alpha):
    n_x=len(x)
    if(1-alpha)*(1+1/n_x)>1:
        return np.inf
    else:
        return np.quantile(x,(1-alpha)*(1+1/n_x))

def weighted_quantile(scores,weights,alpha):
    """
    input:
    scores: n dimensional array
    weights: n dimensional array. Note that scores and weights are one-to-one.
    
    output:
    quantile: a scalar
    
    """
    scores_sorted=np.sort(scores)
    indices=np.argsort(scores)
    weights_sorted=weights[indices]
    cumulative_weights=np.cumsum(weights_sorted)
    index = np.where(cumulative_weights >= 1-alpha)[0][0]
    quantile=scores_sorted[index]
    
    return quantile

    
    

In [5]:
def predict_basemodel(M_candidate_models,optimal_weights,X_test):
    """
    input:
    M_candidate_models: obtained from the function "fit_candidate_model"
    optimal_weights: M dimensional array
    X_test: n*d dimensianal array
    
    output:
    predictions: n dimensional array
    
    """
    M=len(optimal_weights)
    #check if X_test is a single datapoint
    if X_test.ndim==1:
        predictions_M_candidate=np.zeros((M,1))
        for i in range(M):
            X_test_candidate=copy.deepcopy(X_test[1:i+2]).reshape(1,-1)
            predictions_M_candidate[i,]=M_candidate_models[i].predict(X_test_candidate)
            
    else:
        n_test=X_test.shape[0]
        predictions_M_candidate=np.zeros((M,n_test))
        for i in range(M):
            X_test_candidate=copy.deepcopy(X_test[:,1:i+2])
            predictions_M_candidate[i,]=M_candidate_models[i].predict(X_test_candidate)
        
    predictions=predictions_M_candidate.T.dot(optimal_weights)
    
    return predictions

In [6]:
###Functions that compute point prediction error, interval coveerage and length.
def compute_PP_metric(predictions,Y_test):
    """
    Compute the prediction error.
    
    """
    error=np.sum(np.abs(predictions-Y_test))/len(Y_test)
    return error

def compute_PI_metrics(interval_predictions,Y_test):
    """
    Compute interval coverage and length.
    """
    contains = (Y_test <= interval_predictions['y_sup']) & (Y_test >= interval_predictions['y_inf'])
    lengths = interval_predictions['y_sup'] - interval_predictions['y_inf']

    coverage=np.mean(contains)
    len=np.mean(lengths)
    metrics={'cov':coverage,
             'len':len}
    return metrics

In [7]:
##Function that implement SCP_MA
def SCP_MA(M_candidate_models,prediction_jackknife,Y_train,X_cal,Y_cal,X_test,alpha):
    """
    Split conformal prediction for model averaging
    input: 
    M_candidate_model:
    prediction_jackknife:
    Y_train:
    X_cal:
    Y_cal:
    X_test:
    alpha:
    
    output:
    interval_predictions:
    
    """
    Y_train=np.array(Y_train,dtype=np.double)
    JMA_weights=weights_criterion_jackknife(Y_train=Y_train,prediction_jackknife=prediction_jackknife,
                                            kernel_weights=None,candidate_interval_lengths=None,penalty_coef=None,criterion='ordinary')
    pred_cal=predict_basemodel(M_candidate_models,JMA_weights,X_cal)
    scores=np.abs(pred_cal-Y_cal)
    q_scores=quantile_corrected(scores, alpha)
    
    pred_test=predict_basemodel(M_candidate_models,JMA_weights,X_test)
    interval_predictions={'y_inf':pred_test-q_scores,
                          'y_sup':pred_test+q_scores}
    
    return interval_predictions
    
    
    

In [8]:
###Functions that implement SLCP_MA
def compute_candidate_model_lengths(M_candidate_models,X_train,Y_train,X_cal,Y_cal,x_testpoint,h,alpha):
    """
    Compute the prediction length of M candidate models
    input:
    M_candidate_models: 
    X_train:
    Y_train:
    X_cal:
    Y_cal:
    x_testpoint: a single datapoint
    h:
    alpha:
    
    output:
    candidate_model_lengths: M dim
    
    
    """
    M=len(M_candidate_models)
    n_train=len(Y_train)
    n_cal=len(Y_cal)
    candidate_model_lengths=np.zeros(M)
    
    kernel_weights_matrix=np.zeros((n_cal+1,n_train)) #ecah row corresponding to a specific kernel weights
    for i in range(n_cal):
        kernel_weights_matrix[i,]=find_kernel_weights(X_train,X_cal[i,:],h)
    
    kernel_weights_matrix[-1,:]=find_kernel_weights(X_train,x_testpoint,h)
    
    for i in range(M):
        X_train_candidate=copy.deepcopy(X_train[:,1:i+2])
        X_cal_candidate=copy.deepcopy(X_cal[:,1:i+2])
        
        pred_train_candidate=M_candidate_models[i].predict(X_train_candidate)
        pred_cal_candidate=M_candidate_models[i].predict(X_cal_candidate)
        
        scores_train=np.abs(pred_train_candidate-Y_train)
        scores_cal=np.abs(pred_cal_candidate-Y_cal)
        
        ##quantile_weighted:each element corresponding to Q(1-alpha,F_{i}^{h}), for i in [1,...,n+1].
        quantile_weighted=np.zeros(n_cal+1) 
        for j in range(n_cal+1):
            quantile_weighted[j]=weighted_quantile(scores_train,kernel_weights_matrix[j,:],alpha)
        
        V_scores=scores_cal-quantile_weighted[:-1]
        d1=quantile_weighted[-1]+quantile_corrected(V_scores,alpha)
        candidate_model_lengths[i]=2*d1
    
    return candidate_model_lengths
            
def SLCP_MA(M_candidate_models,prediction_jackknife,X_train,Y_train,X_cal,Y_cal,X_test,h,penalty_coef,alpha):
    """
    Split localized conformal prediction for model averaging
    input:
    
    output:
    interval_predictions:
    
    """
    Y_train=np.array(Y_train,dtype=np.double)
    n_train=X_train.shape[0]
    n_cal=X_cal.shape[0]
    n_test=X_test.shape[0]
    
    kernel_weights_matrix=np.zeros((n_cal+1,n_train))
    
    for i in range(n_cal):
        kernel_weights_matrix[i,]=find_kernel_weights(X_train,X_cal[i,],h)
    
    y_inf=np.zeros(n_test)
    y_sup=np.zeros(n_test)
    for i in range(n_test):
        x_testpoint=X_test[i,]
        kernel_weights=find_kernel_weights(X_train,x_testpoint,h)
        kernel_weights_matrix[-1,:]=kernel_weights
        
        candidate_interval_lengths=compute_candidate_model_lengths(M_candidate_models,X_train,Y_train,X_cal,Y_cal,x_testpoint,h,alpha)
        optimal_weights=weights_criterion_jackknife(Y_train,prediction_jackknife,kernel_weights,candidate_interval_lengths,penalty_coef,criterion='weighted_plus_penalty')
        
        pred_train=predict_basemodel(M_candidate_models,optimal_weights,X_train)
        pred_cal=predict_basemodel(M_candidate_models,optimal_weights,X_cal)
        
        scores_train=np.abs(pred_train-Y_train)
        scores_cal=np.abs(pred_cal-Y_cal)
        
        quantile_weighted=np.zeros(n_cal+1)
        for j in range(n_cal+1):
            quantile_weighted[j]=weighted_quantile(scores_train,kernel_weights_matrix[j,:],alpha)
        
        V_scores=scores_cal-quantile_weighted[:-1]
        d1=quantile_weighted[-1]+quantile_corrected(V_scores,alpha)
        
        pred_testpoint=predict_basemodel(M_candidate_models,optimal_weights,x_testpoint)
        y_inf[i]=pred_testpoint-d1
        y_sup[i]=pred_testpoint+d1
    
    interval_predictions={'y_inf':y_inf,
                          'y_sup':y_sup}
    
    return interval_predictions

    

# Simulation studies

## Compare prediction errors of two model averaging methods, and compare coverage and interval length of SCP_MA and SLCP_MA

In [9]:
###setting
n0=50    #training set
n1=50    #calibration set
n=100   #test data
M=int(3*n0**(1/3))
d=50 #The dimensionality of X
R_squared=0.9  ##ranging from 0.1 to 0.9
error='homo' #homo or hetero
h=5 # The bandwidth of Gaussian kernel
penalty_coef=1.0  #penalty term
alpha=0.1
replication=1

mean0=0
mean1=0.3


JMA_risk=np.zeros(replication)
weighted_JMA_risk=np.zeros(replication)

SCP_MA_cov=np.zeros(replication)
SCP_MA_len=np.zeros(replication)
SLCP_MA_cov=np.zeros(replication)
SLCP_MA_len=np.zeros(replication)

for k in range(replication):
    df=generate_data(n0+n1+n,d,R_squared,error,mean=mean0)
    df1=split_data(df,n0,n1,n,d_set='train')
    df2=split_data(df,n0,n1,n,d_set='cal')
    
    #df=generate_data(n0+n1+n,d,R_squared,error,mean=mean1)
    df3=split_data(df,n0,n1,n,d_set='test') 
    
    #train M candidate models
    M_candidate_models=fit_candidate_model(df1['X'],df1['Y'],M)
    
    prediction_jackknife=jackknife_prediction(df1['X'],df1['Y'],M)
    
    Y_train=np.array(df1['Y'],dtype=np.double)
    
    ##################################First: compare their test errors--------------------------------------------
    ##JMA
    JMA_weights=weights_criterion_jackknife(Y_train=Y_train,prediction_jackknife=prediction_jackknife,
                                            kernel_weights=None,candidate_interval_lengths=None,penalty_coef=None,criterion='ordinary')
    JMA_predictions=predict_basemodel(M_candidate_models,JMA_weights,df3['X'])
    
    ##weighted_JMA
    weighted_JMA_predictions=np.zeros(n)
    for i in range(n):
        kernel_weights=find_kernel_weights(X=df1['X'],x_datapoint=df3['X'][i,],h=h)
        weighted_JMA_weights=weights_criterion_jackknife(Y_train=Y_train,prediction_jackknife=prediction_jackknife,
                                                         kernel_weights=kernel_weights,candidate_interval_lengths=None,penalty_coef=None,criterion='weighted')
        weighted_JMA_predictions[i]=predict_basemodel(M_candidate_models,weighted_JMA_weights,df3['X'][i,:])   
    
    ###compute their errors
    JMA_risk[k]=compute_PP_metric(predictions=JMA_predictions,Y_test=df3['Y'])
    weighted_JMA_risk[k]=compute_PP_metric(predictions=weighted_JMA_predictions,Y_test=df3['Y'])
    
    
    
    ##################################Second: compare their coverage and interval length--------------------------------------------
    SCP_MA_pred_intervals=SCP_MA(M_candidate_models,prediction_jackknife,df2['Y'],df2['X'],df2['Y'],df3['X'],alpha)
    
    SLCP_MA_pred_intervals=SLCP_MA(M_candidate_models,prediction_jackknife,df1['X'],df1['Y'],df2['X'],df2['Y'],df3['X'],h,penalty_coef,alpha)
    
    
    #####
    SCP_MA_metrics=compute_PI_metrics(SCP_MA_pred_intervals,df3['Y'])
    SLCP_MA_metrics=compute_PI_metrics(SLCP_MA_pred_intervals,df3['Y'])
    
    ###
    SCP_MA_cov[k]=SCP_MA_metrics['cov']
    SCP_MA_len[k]=SCP_MA_metrics['len']
    SLCP_MA_cov[k]=SLCP_MA_metrics['cov']
    SLCP_MA_len[k]=SLCP_MA_metrics['len']
    
    
    print(k)
    

0


In [10]:
#print("JMA and weighted_JMA prediction error: : ", round(np.mean(JMA_risk),3),round(np.mean(weighted_JMA_risk),3))
#print("SCP_MA: coverage and length: ", round(np.mean(SCP_MA_cov),3),round(np.mean(SCP_MA_len),3))
#print("SLCP_MA: coverage and length: ", round(np.mean(SLCP_MA_cov),3),round(np.mean(SLCP_MA_len),3))

In [11]:
print("JMA and weighted_JMA prediction error: : ", round(np.mean(JMA_risk),3),round(np.mean(weighted_JMA_risk),3))
print("coverage: ", round(np.mean(SCP_MA_cov),3),round(np.mean(SLCP_MA_cov),3))
print("length: ", round(np.mean(SCP_MA_len),3),round(np.mean(SLCP_MA_len),3))

JMA and weighted_JMA prediction error: :  1.42 1.431
coverage:  0.87 0.76
length:  8.615 4.163


# Run the following code

In [23]:
R_squared_vec=np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])  ##ranging from 0.1 to 0.9
num_R_squared=len(R_squared_vec)

JMA_risk_vec=np.zeros(num_R_squared)
weighted_JMA_risk_vec=np.zeros(num_R_squared)
SCP_cov_vec=np.zeros(num_R_squared)
SCP_len_vec=np.zeros(num_R_squared)
SLCP_cov_vec=np.zeros(num_R_squared)
SLCP_len_vec=np.zeros(num_R_squared)


for t in range(num_R_squared):
    n0=50*3    #training set
    n1=50*3    #calibration set
    n=100   #test data
    M=int(3*n0**(1/3))
    d=50 #The dimension of X

    error='homo' #homo or hetero
    h=1 # The bandwidth of Gaussian kernel
    penalty_coef=1  #penalty term
    alpha=0.1
    replication=15
    R_squared=R_squared_vec[t]  ##ranging from 0.1 to 0.9
    mean0=0
    #mean1=0.3


    JMA_risk=np.zeros(replication)
    weighted_JMA_risk=np.zeros(replication)

    SCP_MA_cov=np.zeros(replication)
    SCP_MA_len=np.zeros(replication)
    SLCP_MA_cov=np.zeros(replication)
    SLCP_MA_len=np.zeros(replication)

    for k in range(replication):
        df=generate_data(n0+n1+n,d,R_squared,error,mean=mean0)
        df1=split_data(df,n0,n1,n,d_set='train')
        df2=split_data(df,n0,n1,n,d_set='cal')

        #df=generate_data(n0+n1+n,d,R_squared,error,mean=mean1)
        df3=split_data(df,n0,n1,n,d_set='test') 

        #train M candidate models
        M_candidate_models=fit_candidate_model(df1['X'],df1['Y'],M)

        prediction_jackknife=jackknife_prediction(df1['X'],df1['Y'],M)

        Y_train=np.array(df1['Y'],dtype=np.double)

        ##################################First: compare their test errors--------------------------------------------
        ##JMA
        JMA_weights=weights_criterion_jackknife(Y_train=Y_train,prediction_jackknife=prediction_jackknife,
                                                kernel_weights=None,candidate_interval_lengths=None,penalty_coef=None,criterion='ordinary')
        JMA_predictions=predict_basemodel(M_candidate_models,JMA_weights,df3['X'])

        ##weighted_JMA
        weighted_JMA_predictions=np.zeros(n)
        for i in range(n):
            kernel_weights=find_kernel_weights(X=df1['X'],x_datapoint=df3['X'][i,],h=h)
            weighted_JMA_weights=weights_criterion_jackknife(Y_train=Y_train,prediction_jackknife=prediction_jackknife,
                                                             kernel_weights=kernel_weights,candidate_interval_lengths=None,penalty_coef=None,criterion='weighted')
            weighted_JMA_predictions[i]=predict_basemodel(M_candidate_models,weighted_JMA_weights,df3['X'][i,:])   

        ###compute their errors
        JMA_risk[k]=compute_PP_metric(predictions=JMA_predictions,Y_test=df3['Y'])
        weighted_JMA_risk[k]=compute_PP_metric(predictions=weighted_JMA_predictions,Y_test=df3['Y'])



        ##################################Second: compare their coverage and interval length--------------------------------------------
        SCP_MA_pred_intervals=SCP_MA(M_candidate_models,prediction_jackknife,df2['Y'],df2['X'],df2['Y'],df3['X'],alpha)

        SLCP_MA_pred_intervals=SLCP_MA(M_candidate_models,prediction_jackknife,df1['X'],df1['Y'],df2['X'],df2['Y'],df3['X'],h,penalty_coef,alpha)


        #####
        SCP_MA_metrics=compute_PI_metrics(SCP_MA_pred_intervals,df3['Y'])
        SLCP_MA_metrics=compute_PI_metrics(SLCP_MA_pred_intervals,df3['Y'])

        ###
        SCP_MA_cov[k]=SCP_MA_metrics['cov']
        SCP_MA_len[k]=SCP_MA_metrics['len']
        SLCP_MA_cov[k]=SLCP_MA_metrics['cov']
        SLCP_MA_len[k]=SLCP_MA_metrics['len']
    
    ######
    JMA_risk_vec[t]=np.mean(JMA_risk)
    weighted_JMA_risk_vec[t]=np.mean(weighted_JMA_risk)
    SCP_cov_vec[t]=np.mean(SCP_MA_cov)
    SCP_len_vec[t]=np.mean(SCP_MA_len)
    SLCP_cov_vec[t]=np.mean(SLCP_MA_cov)
    SLCP_len_vec[t]=np.mean(SLCP_MA_len)
    print(t)
    
    



0
1
2
3
4
5
6
7
8


In [24]:
print("JMA risk: ", np.round(JMA_risk_vec,3))
print("weighted JMA risk: ", np.round(weighted_JMA_risk_vec,3))
print("SCP coverage: ", np.round(SCP_cov_vec,3))
print("SLCP coverage: ", np.round(SLCP_cov_vec,3))
print("SCP interval length: ", np.round(SCP_len_vec,3))
print("SLCP interval length: ", np.round(SLCP_len_vec,3))

JMA risk:  [0.826 0.843 0.814 0.842 0.858 0.866 0.883 0.927 1.131]
weighted JMA risk:  [0.841 0.859 0.827 0.871 0.922 0.899 0.956 0.996 1.273]
SCP coverage:  [0.901 0.891 0.913 0.902 0.908 0.914 0.912 0.907 0.905]
SLCP coverage:  [0.899 0.881 0.889 0.898 0.9   0.881 0.886 0.884 0.875]
SCP interval length:  [3.399 3.598 3.744 3.826 4.163 4.541 4.937 5.752 8.277]
SLCP interval length:  [3.815 3.801 3.825 4.02  3.942 3.891 4.035 4.2   4.975]


In [25]:
import winsound

print("代码已运行完毕")

# 播放系统提示音
winsound.Beep(440, 2000)  # 第一个参数是频率（以赫兹为单位），第二个参数是持续时间（以毫秒为单位）

代码已运行完毕
